# Chapter 17: Shortest Path Algorithms

> *"The shortest path is not just about distance—it's about efficiency, cost, and the optimal way to connect points in a network."* — Anonymous

---

## 17.1 Introduction to Shortest Path Problems

Finding the shortest path between vertices in a graph is one of the most fundamental and widely applicable problems in computer science. It underpins everything from GPS navigation and network routing to social network analysis and game development.

### 17.1.1 Problem Variants

Shortest path problems come in several flavors:

- **Single-Source Shortest Path (SSSP):** Find shortest paths from a given source vertex to all other vertices.
- **Single-Destination Shortest Path:** Find shortest paths from all vertices to a given destination (equivalent to SSSP on reversed graph).
- **All-Pairs Shortest Path (APSP):** Find shortest paths between every pair of vertices.

### 17.1.2 Graph Characteristics

The choice of algorithm depends heavily on graph properties:

- **Unweighted graphs:** BFS solves SSSP in O(V+E).
- **Weighted graphs without negative edges:** Dijkstra's algorithm is the classic choice.
- **Graphs with negative weights (but no negative cycles):** Bellman-Ford or SPFA.
- **Graphs with negative cycles:** Detectable but no finite shortest path exists (paths can be arbitrarily negative).
- **Dense graphs vs sparse graphs:** Floyd-Warshall (O(V³)) works for dense graphs; Johnson's algorithm (O(VE + V² log V)) is better for sparse all-pairs.

### 17.1.3 Why It Matters

```
┌─────────────────────────────────────────────────────────────────────┐
│                    IMPORTANCE OF SHORTEST PATHS                      │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  1. GPS NAVIGATION: Find fastest route between locations            │
│  2. NETWORK ROUTING: OSPF, RIP protocols use shortest path algorithms│
│  3. SOCIAL NETWORKS: Degrees of separation (shortest path in       │
│     unweighted graph)                                                │
│  4. TRANSPORTATION: Optimize delivery routes, airline schedules     │
│  5. GAME DEVELOPMENT: Pathfinding for AI characters (A*)            │
│  6. TELECOMMUNICATIONS: Minimize latency in network design          │
│  7. BIOLOGY: Find shortest pathways in metabolic networks           │
│  8. FINANCE: Arbitrage detection (negative cycles in currency graphs)│
│  9. ROBOTICS: Motion planning in configuration space                │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 17.2 Dijkstra's Algorithm

Dijkstra's algorithm solves the single-source shortest path problem for graphs with **non-negative edge weights**. It is a greedy algorithm that progressively expands the frontier of known shortest distances.

### 17.2.1 Intuition

Dijkstra's algorithm maintains a set of vertices whose shortest distance from the source is already determined. At each step, it selects the vertex with the smallest tentative distance among the unprocessed vertices, relaxes its outgoing edges, and repeats.

### 17.2.2 Algorithm (Lazy Implementation with Priority Queue)

```python
import heapq
from typing import List, Dict, Tuple, Optional

def dijkstra_lazy(graph: List[List[Tuple[int, int]]], source: int) -> Tuple[List[float], List[Optional[int]]]:
    """
    Dijkstra's algorithm (lazy version using priority queue).
    
    Args:
        graph: adjacency list where graph[u] = [(v, weight), ...]
        source: source vertex index
    
    Returns:
        dist: list of shortest distances from source
        prev: list of previous vertices for path reconstruction
    """
    n = len(graph)
    dist = [float('inf')] * n
    prev = [None] * n
    dist[source] = 0
    
    # Priority queue entries: (distance, vertex)
    pq = [(0, source)]
    visited = [False] * n
    
    while pq:
        d, u = heapq.heappop(pq)
        if visited[u]:
            continue  # already processed (lazy deletion)
        visited[u] = True
        
        for v, w in graph[u]:
            if not visited[v] and dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                prev[v] = u
                heapq.heappush(pq, (dist[v], v))
    
    return dist, prev
```

### 17.2.3 Complexity

- **Time:** O((V + E) log V) with a binary heap. Each vertex is popped once, each edge considered once, and heap operations are O(log V).
- **Space:** O(V) for distances and priority queue.

### 17.2.4 Proof of Correctness

The algorithm relies on the invariant that when a vertex is popped from the priority queue with the smallest distance, that distance is final because all edges have non-negative weights. Any alternative path would have to go through vertices with larger distances, making it longer.

### 17.2.5 Eager Implementation (with Decrease-Key)

For better performance, we can maintain an indexable priority queue that supports decrease-key operations. This avoids multiple entries for the same vertex.

```python
class IndexedMinHeap:
    def __init__(self, n):
        self.n = n
        self.values = [float('inf')] * n
        self.pos = [-1] * n
        self.heap = []  # list of (value, vertex)
    
    def push_or_decrease(self, vertex, value):
        if self.pos[vertex] == -1:
            # Not in heap
            heapq.heappush(self.heap, (value, vertex))
            self.pos[vertex] = len(self.heap) - 1
            self.values[vertex] = value
        elif value < self.values[vertex]:
            # Decrease key
            self.values[vertex] = value
            # Bubble up
            i = self.pos[vertex]
            while i > 0 and self.heap[i][0] < self.heap[(i-1)//2][0]:
                self.heap[i], self.heap[(i-1)//2] = self.heap[(i-1)//2], self.heap[i]
                self.pos[self.heap[i][1]] = i
                self.pos[self.heap[(i-1)//2][1]] = (i-1)//2
                i = (i-1)//2
    
    def pop_min(self):
        if not self.heap:
            return None
        value, vertex = heapq.heappop(self.heap)
        self.pos[vertex] = -1
        # Update pos for moved element
        if self.heap:
            self.pos[self.heap[0][1]] = 0
        return vertex, value

def dijkstra_eager(graph, source):
    n = len(graph)
    dist = [float('inf')] * n
    prev = [None] * n
    dist[source] = 0
    pq = IndexedMinHeap(n)
    pq.push_or_decrease(source, 0)
    
    while True:
        u = pq.pop_min()
        if u is None:
            break
        u_vertex, u_dist = u
        if u_dist > dist[u_vertex]:
            continue  # stale entry
        for v, w in graph[u_vertex]:
            if dist[u_vertex] + w < dist[v]:
                dist[v] = dist[u_vertex] + w
                prev[v] = u_vertex
                pq.push_or_decrease(v, dist[v])
    return dist, prev
```

### 17.2.6 Example

Consider a graph with vertices A=0, B=1, C=2, D=3, E=4, and edges:
- 0→1 (4), 0→2 (2)
- 1→2 (1), 1→3 (5)
- 2→3 (8), 2→4 (10)
- 3→4 (2)

Running Dijkstra from 0 yields:
- dist = [0, 4, 2, 9, 11]
- prev = [None, 0, 0, 1, 3]

### 17.2.7 Limitations

- **Negative edges:** Dijkstra fails because a later vertex with a smaller distance might still be improved by a negative edge, but it would have already been finalized.
- **Negative cycles:** Not applicable (non-negative weights).

### 17.2.8 Bidirectional Dijkstra

For single-source single-target queries, we can run two Dijkstra searches simultaneously: one forward from source, one backward from target. When the frontiers meet, we can reconstruct the shortest path. This can significantly reduce search space.

---

## 17.3 Bellman-Ford Algorithm

Bellman-Ford solves the single-source shortest path problem even in the presence of negative edge weights, provided there are no negative cycles reachable from the source. It can also detect negative cycles.

### 17.3.1 Intuition

The algorithm repeatedly relaxes all edges, gradually improving distance estimates. After |V|-1 iterations, all shortest paths (which can have at most |V|-1 edges) are found. An additional iteration checks for negative cycles.

### 17.3.2 Algorithm

```python
def bellman_ford(graph: List[List[Tuple[int, int]]], source: int) -> Tuple[List[float], List[Optional[int]], bool]:
    """
    Bellman-Ford algorithm.
    
    Returns:
        dist: shortest distances from source
        prev: previous vertices for path reconstruction
        has_negative_cycle: True if negative cycle reachable from source
    """
    n = len(graph)
    dist = [float('inf')] * n
    prev = [None] * n
    dist[source] = 0
    
    # Relax edges |V|-1 times
    for _ in range(n - 1):
        updated = False
        for u in range(n):
            if dist[u] == float('inf'):
                continue
            for v, w in graph[u]:
                if dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w
                    prev[v] = u
                    updated = True
        if not updated:
            break  # early termination if no improvement
    
    # Check for negative cycles
    for u in range(n):
        if dist[u] == float('inf'):
            continue
        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                return dist, prev, True  # negative cycle detected
    
    return dist, prev, False
```

### 17.3.3 Complexity

- **Time:** O(V × E) – each iteration processes all edges.
- **Space:** O(V) for distances.

### 17.3.4 Example

Graph with negative weights:
- 0→1 (5)
- 1→2 (2)
- 2→3 (-4)
- 3→1 (1)

Running from 0:
Iteration 1: dist[1]=5, dist[2]=7, dist[3]=3, dist[1] improves to 4 via 3? Wait, path 0-1-2-3-1: 5+2-4+1=4, so after enough iterations, distances converge. No negative cycle because cycle 1-2-3-1 has total weight 2-4+1 = -1? Actually that's negative! Let's check: 1→2 (2), 2→3 (-4), 3→1 (1) sum = -1 → negative cycle. After |V|-1 iterations, further relaxation still improves, so algorithm detects negative cycle.

### 17.3.5 Detecting Negative Cycles

If after V-1 iterations we can still relax an edge, a negative cycle reachable from the source exists. In such case, shortest paths are not well-defined (they can be arbitrarily negative).

### 17.3.6 Applications

- **Currency arbitrage:** Model currencies as vertices, exchange rates as edges (logarithms to convert multiplication to addition), find negative cycles representing arbitrage opportunities.
- **Routing protocols** like RIP (Routing Information Protocol) use a variant (distance-vector routing) that is susceptible to count-to-infinity problems, mitigated by techniques like split horizon.

---

## 17.4 SPFA (Shortest Path Faster Algorithm)

SPFA is a queue-based optimization of Bellman-Ford that often runs faster in practice, though its worst-case complexity remains O(VE).

### 17.4.1 Intuition

Instead of relaxing all edges blindly V-1 times, SPFA maintains a queue of vertices whose distance has improved. Only edges from those vertices are relaxed, potentially reducing the number of relaxations.

### 17.4.2 Algorithm

```python
from collections import deque

def spfa(graph: List[List[Tuple[int, int]]], source: int) -> Tuple[List[float], List[Optional[int]], bool]:
    n = len(graph)
    dist = [float('inf')] * n
    prev = [None] * n
    in_queue = [False] * n
    count = [0] * n  # number of times vertex has been updated
    dist[source] = 0
    queue = deque([source])
    in_queue[source] = True
    
    while queue:
        u = queue.popleft()
        in_queue[u] = False
        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                prev[v] = u
                if not in_queue[v]:
                    queue.append(v)
                    in_queue[v] = True
                    count[v] += 1
                    if count[v] >= n:  # negative cycle detected
                        return dist, prev, True
    return dist, prev, False
```

### 17.4.3 Complexity

- **Average case:** O(E) – very fast on many graphs.
- **Worst case:** O(VE) – can be forced by adversarial inputs (e.g., complete graph with carefully chosen edge weights).

### 17.4.4 When to Use

SPFA is simple to implement and often performs well on real-world graphs. However, because of its worst-case behavior, Dijkstra is preferred for graphs without negative weights, and Bellman-Ford's predictable O(VE) may be safer for critical applications.

---

## 17.5 Floyd-Warshall Algorithm

Floyd-Warshall solves the **all-pairs shortest path** problem using dynamic programming. It works for graphs with negative edges (but no negative cycles) and also computes the transitive closure.

### 17.5.1 Intuition

Let `dist[k][i][j]` be the shortest path from i to j using only intermediate vertices from the set {0,1,…,k}. The recurrence:
```
dist[k][i][j] = min(dist[k-1][i][j], dist[k-1][i][k] + dist[k-1][k][j])
```
We can implement it in-place using a 2D matrix, iterating k from 0 to V-1.

### 17.5.2 Algorithm

```python
def floyd_warshall(weights: List[List[float]]) -> Tuple[List[List[float]], List[List[Optional[int]]]]:
    """
    Floyd-Warshall all-pairs shortest paths.
    
    Args:
        weights: adjacency matrix where weights[i][j] is edge weight,
                 or float('inf') if no edge; weights[i][i] = 0.
    
    Returns:
        dist: matrix of shortest distances
        next: matrix for path reconstruction (next[i][j] is next vertex after i)
    """
    V = len(weights)
    dist = [[weights[i][j] for j in range(V)] for i in range(V)]
    next_vertex = [[None if weights[i][j] == float('inf') else j for j in range(V)] for i in range(V)]
    
    for k in range(V):
        for i in range(V):
            if dist[i][k] == float('inf'):
                continue
            for j in range(V):
                if dist[k][j] == float('inf'):
                    continue
                if dist[i][k] + dist[k][j] < dist[i][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    next_vertex[i][j] = next_vertex[i][k]
    
    # Check for negative cycles (if any diagonal becomes negative)
    for i in range(V):
        if dist[i][i] < 0:
            return dist, next_vertex, True  # negative cycle detected
    
    return dist, next_vertex, False
```

### 17.5.3 Path Reconstruction

Using the `next` matrix:

```python
def reconstruct_path_floyd(next_matrix, u, v):
    if next_matrix[u][v] is None:
        return None
    path = [u]
    while u != v:
        u = next_matrix[u][v]
        path.append(u)
    return path
```

### 17.5.4 Complexity

- **Time:** O(V³) – three nested loops.
- **Space:** O(V²) for the distance matrix.

### 17.5.5 Applications

- **Finding transitive closure** (by using boolean adjacency matrix).
- **Detecting negative cycles** (if diagonal becomes negative after algorithm).
- **Small graphs** where V ≤ 500 typically.

---

## 17.6 Johnson's Algorithm

Johnson's algorithm combines Bellman-Ford and Dijkstra to efficiently compute all-pairs shortest paths in sparse graphs, even with negative weights (but no negative cycles). It runs in O(VE + V² log V), which is better than Floyd-Warshall for sparse graphs.

### 17.6.1 Intuition

If there were no negative edges, we could run Dijkstra from each vertex (V times) for O(V² log V + VE). Negative edges prevent direct use of Dijkstra. Johnson's idea: reweight the graph using a potential function to make all edge weights non-negative, then run Dijkstra.

### 17.6.2 Algorithm Steps

1. Add a new vertex s with zero-weight edges to all original vertices.
2. Run Bellman-Ford from s to compute potentials h(v) (shortest distances from s). If negative cycle detected, terminate.
3. Reweight each edge (u→v) with weight `w'(u,v) = w(u,v) + h(u) - h(v)`. This ensures all new weights are non-negative.
4. For each original vertex u, run Dijkstra using the reweighted graph to compute distances d'(u, v).
5. Convert back: original distance d(u,v) = d'(u,v) - h(u) + h(v).

### 17.6.3 Implementation

```python
def johnson(graph: List[List[Tuple[int, int]]]) -> Tuple[List[List[float]], bool]:
    """
    Johnson's algorithm for all-pairs shortest paths.
    
    Args:
        graph: adjacency list of original graph (weights may be negative)
    
    Returns:
        dist: matrix of shortest distances (float('inf') if unreachable)
        has_negative_cycle: True if graph contains negative cycle
    """
    V = len(graph)
    # Add new vertex 0 (index V) with edges to all others weight 0
    augmented_graph = graph + [[]]
    for i in range(V):
        augmented_graph[V].append((i, 0))
    
    # Bellman-Ford from new vertex
    h, _, has_neg_cycle = bellman_ford(augmented_graph, V)
    if has_neg_cycle:
        return None, True
    
    # Reweight original graph
    new_graph = [[] for _ in range(V)]
    for u in range(V):
        for v, w in graph[u]:
            new_graph[u].append((v, w + h[u] - h[v]))
    
    # Run Dijkstra from each vertex
    dist = [[float('inf')] * V for _ in range(V)]
    for u in range(V):
        d, _ = dijkstra_lazy(new_graph, u)
        for v in range(V):
            if d[v] < float('inf'):
                dist[u][v] = d[v] - h[u] + h[v]
            else:
                dist[u][v] = float('inf')
    
    return dist, False
```

### 17.6.4 Complexity

- **Bellman-Ford:** O(VE)
- **V times Dijkstra:** O(V (V+E) log V) = O(V² log V + VE log V) – often simplified as O(VE + V² log V) because log factors matter.
- **Overall:** Dominated by Dijkstra runs.

### 17.6.5 When to Use

Johnson's algorithm is ideal for sparse graphs (E ≈ V) with negative weights, where Floyd-Warshall's O(V³) would be too slow.

---

## 17.7 Specialized Shortest Path Algorithms

### 17.7.1 0-1 BFS

When edge weights are only 0 or 1, we can achieve O(V+E) using a deque (double-ended queue) instead of a priority queue.

```python
from collections import deque

def zero_one_bfs(graph: List[List[Tuple[int, int]]], source: int) -> List[float]:
    """
    graph: adjacency list where weights are 0 or 1.
    """
    n = len(graph)
    dist = [float('inf')] * n
    dist[source] = 0
    dq = deque([source])
    
    while dq:
        u = dq.popleft()
        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                if w == 0:
                    dq.appendleft(v)
                else:
                    dq.append(v)
    return dist
```

**Intuition:** Nodes reached via 0-weight edges are placed at front of deque for immediate processing, preserving BFS-like order.

### 17.7.2 Dial's Algorithm

For integer weights in a small range [0, C], we can use an array of buckets (like counting sort) to implement a priority queue with O(V+E + C) time.

```python
def dials_algorithm(graph: List[List[Tuple[int, int]]], source: int, max_weight: int):
    n = len(graph)
    dist = [float('inf')] * n
    dist[source] = 0
    # Create buckets (list of deques) for distances 0..max_weight * (n-1) roughly
    # But we can use modulo to handle larger distances
    # Simplified version: use dictionary of queues keyed by distance.
    # For efficiency, we can use an array of size n * max_weight + 1.
    max_dist = n * max_weight
    buckets = [deque() for _ in range(max_dist + 1)]
    buckets[0].append(source)
    current_dist = 0
    
    while current_dist <= max_dist:
        while buckets[current_dist]:
            u = buckets[current_dist].popleft()
            if current_dist > dist[u]:
                continue
            for v, w in graph[u]:
                new_dist = dist[u] + w
                if new_dist < dist[v]:
                    dist[v] = new_dist
                    buckets[new_dist].append(v)
        current_dist += 1
    return dist
```

**Note:** This uses O(CV) buckets, which may be large. Optimizations use modulo with `(current_dist + w) % (n * max_weight)` but careful handling required.

### 17.7.3 A* Search Algorithm

A* is a heuristic-guided search algorithm that finds the shortest path from a source to a **single target** more efficiently than Dijkstra when a good heuristic (admissible and consistent) is available. It is widely used in games and robotics.

A* maintains, for each vertex v, a value `f(v) = g(v) + h(v)`, where:
- `g(v)` is the cost from source to v (as in Dijkstra).
- `h(v)` is a heuristic estimate of the cost from v to the target (must be admissible, i.e., never overestimate true cost).

```python
import heapq

def a_star(graph, source, target, heuristic):
    """
    graph: adjacency list (u -> (v, weight))
    heuristic: function(v) returns estimated cost from v to target
    """
    n = len(graph)
    g = [float('inf')] * n
    g[source] = 0
    f = [float('inf')] * n
    f[source] = heuristic(source)
    prev = [None] * n
    open_set = [(f[source], source)]
    closed_set = set()
    
    while open_set:
        _, u = heapq.heappop(open_set)
        if u == target:
            # Reconstruct path
            path = []
            while u is not None:
                path.append(u)
                u = prev[u]
            return path[::-1], g[target]
        closed_set.add(u)
        for v, w in graph[u]:
            if v in closed_set:
                continue
            tentative_g = g[u] + w
            if tentative_g < g[v]:
                prev[v] = u
                g[v] = tentative_g
                f[v] = g[v] + heuristic(v)
                heapq.heappush(open_set, (f[v], v))
    return None, float('inf')
```

**Properties:**
- With an admissible heuristic, A* is optimal.
- With a consistent (monotone) heuristic, A* never re-opens nodes and is as efficient as possible.

---

## 17.8 Comparison and Selection Guide

```
┌──────────────────────┬──────────────┬───────────────┬────────────────────┐
│ Algorithm            │ Time         │ Space         │ Use Case           │
├──────────────────────┼──────────────┼───────────────┼────────────────────┤
│ Dijkstra (binary heap)│ O((V+E)log V)│ O(V)          │ SSSP, non-negative │
│ Bellman-Ford         │ O(VE)        │ O(V)          │ SSSP, negative     │
│ SPFA                 │ O(E) avg,    │ O(V)          │ SSSP, negative     │
│                      │ O(VE) worst  │               │ (risk of worst)    │
│ Floyd-Warshall       │ O(V³)        │ O(V²)         │ APSP, dense graphs │
│ Johnson              │ O(VE + V² log V)│ O(V²)       │ APSP, sparse, negative │
│ 0-1 BFS              │ O(V+E)       │ O(V)          │ weights 0/1        │
│ Dial                 │ O(V+E+C)     │ O(CV)         │ small integer weights │
│ A*                   │ O(E) in practice│ O(V)        │ source-target, heuristic│
└──────────────────────┴──────────────┴───────────────┴────────────────────┘
```

**Guidelines:**

- **Unweighted:** BFS (not covered here but O(V+E)).
- **Non-negative weights, single source:** Dijkstra.
- **Negative weights, single source, need detection:** Bellman-Ford (or SPFA cautiously).
- **All-pairs, dense graph:** Floyd-Warshall.
- **All-pairs, sparse graph, negative weights:** Johnson.
- **Very small graphs:** Floyd-Warshall for simplicity.
- **0/1 weights:** 0-1 BFS.
- **Small integer weights:** Dial's algorithm.
- **Point-to-point with good heuristic:** A*.

---

## 17.9 Applications of Shortest Paths

1. **GPS Navigation:** Dijkstra or A* on road networks (with heuristics like Euclidean distance).
2. **Network Routing Protocols:** OSPF (Open Shortest Path First) uses Dijkstra; RIP uses distance-vector (Bellman-Ford variant).
3. **Social Networks:** Shortest path (unweighted) gives degrees of separation.
4. **Flight Scheduling:** Find cheapest or fastest itinerary (may involve negative weights for layovers).
5. **Currency Arbitrage:** Detect negative cycles in currency exchange graphs using Bellman-Ford.
6. **Robotics Path Planning:** A* on grid or configuration space.
7. **Game Development:** Pathfinding for characters (A* with Manhattan or Euclidean heuristics).
8. **Transportation Logistics:** Optimize delivery routes with time windows (often extended with constraints).
9. **Telecommunications:** Minimize latency in network design.

---

## 17.10 Practice Problems

### Problem 1: Network Delay Time (LeetCode 743)
You are given a network of n nodes and a list of times (u → v, t). Find the time it takes for all nodes to receive a signal sent from a given node k. Return -1 if not all nodes reachable.

**Hint:** Single-source shortest path (Dijkstra). Return max of distances.

### Problem 2: Cheapest Flights Within K Stops (LeetCode 787)
Find cheapest price from src to dst with at most k stops. Graph may have negative weights? Usually not. Use Bellman-Ford with k+1 iterations (relax edges only k+1 times) to respect stop count.

### Problem 3: Path with Maximum Probability (LeetCode 1514)
Given undirected weighted graph with probabilities on edges, find path with maximum product of probabilities from start to end.

**Hint:** Transform to shortest path by taking logarithms (negative log). Dijkstra works.

### Problem 4: Find the City With the Smallest Number of Neighbors at a Threshold Distance (LeetCode 1334)
Given weighted graph and threshold distance, for each city find number of reachable cities within threshold, return city with smallest count (and largest index tie-break).

**Hint:** All-pairs shortest paths (Floyd-Warshall) then count.

### Problem 5: Minimum Cost to Make at Least One Valid Path in a Grid (LeetCode 1368)
Grid with arrows (cost 0 to follow arrow, cost 1 to change direction). Find min cost to go from top-left to bottom-right.

**Hint:** 0-1 BFS.

### Problem 6: Word Ladder II (LeetCode 126)
Find all shortest transformation sequences from beginWord to endWord.

**Hint:** BFS to find distances, then DFS/backtracking to reconstruct paths.

### Problem 7: Alien Dictionary (LeetCode 269)
Given a sorted dictionary, find order of characters. Build graph, topological sort (Kahn). If cycle exists, invalid.

**Hint:** This is not directly shortest path, but uses graph traversal.

### Problem 8: Find the Safest Path in a Grid (Similar to LeetCode 778)
Given grid with values, find path minimizing maximum value along path (minimax path). Can be solved with Dijkstra variant using max of edge weights.

### Problem 9: Shortest Path in Grid with Obstacles Elimination (LeetCode 1293)
Find shortest path from top-left to bottom-right where you can eliminate at most k obstacles. BFS with state (i, j, k).

### Problem 10: Bus Routes (LeetCode 815)
Given bus routes (list of stops), find minimum number of buses to go from source to target. Model as graph where stops are vertices and buses connect all stops on same route (with weight 1 if boarding a bus). Then BFS.

---

## 17.11 Further Reading

1. **"Introduction to Algorithms" (CLRS)** – Chapter 24 (Single-Source Shortest Paths), Chapter 25 (All-Pairs Shortest Paths)
2. **"Algorithms"** by Robert Sedgewick – Chapter 4 (Shortest Paths)
3. **"The Algorithm Design Manual"** by Steven Skiena – Chapter 6 (Weighted Graph Algorithms)
4. **"Network Flows: Theory, Algorithms, and Applications"** by Ahuja, Magnanti, Orli – In-depth coverage of shortest paths and related topics
5. **Original Papers**:
   - Dijkstra, E. W. (1959) – "A note on two problems in connexion with graphs"
   - Bellman, R. (1958) – "On a routing problem"
   - Floyd, R. W. (1962) – "Algorithm 97: Shortest Path"
   - Johnson, D. B. (1977) – "Efficient algorithms for shortest paths in sparse networks"
   - Hart, P. E., Nilsson, N. J., & Raphael, B. (1968) – "A formal basis for the heuristic determination of minimum cost paths" (A*)

---

> **Coming in Chapter 18**: **Minimum Spanning Trees** – We'll explore algorithms for finding the cheapest way to connect all vertices in a graph.

---

**End of Chapter 17**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='16. graph_traversals.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='18. minimum_spanning_trees.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
